In [1]:
import pandas as pd
import numpy as np
import pickle

# Load ML Pkgs
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

# Transformers
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# Others
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,plot_confusion_matrix

from sklearn.multioutput import MultiOutputClassifier

df = pd.read_csv("train.txt",header=None, sep=";", names=["Comment","Emotion"])

import nltk
import unicodedata
from nltk.corpus import stopwords
import re

In [2]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = re.sub(r"text+","",w) #removing "text" from every sentence
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

In [3]:
#CLEANING SENTENCES
sentences = df['Comment']
sentences = sentences.astype(str)
sentences = sentences.map(preprocess_sentence)
df['Comment'] = sentences

# Features & Labels
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna("NA", inplace=True)

Xfeatures = df['Comment']
ylabels = df[['Emotion']]

# Split Data
x_train,x_test,y_train,y_test = train_test_split(Xfeatures,ylabels,test_size=0.1,random_state=30)

pipe_lr = Pipeline(steps=[('cv',CountVectorizer()),
                          ('lr_multi',LogisticRegression())])

In [4]:
pipe_lr.fit(x_train,y_train)

def listToString(s):
    str1 = ""
    for ele in s:
        str1 += ele

    return str1
def predict(x):
    y = pipe_lr.predict([x])
    z = listToString(y)
    print(z)

c:\Python39\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
#pickle_out = open("classifier.pkl","wb")
#pickle.dump(pipe_lr, pickle_out)
#pickle_out.close()

In [8]:
import joblib
joblib.dump(pipe_lr, "emotion.pkl")

['emotion.pkl']

In [9]:
m = joblib.load("emotion.pkl")

In [10]:
m.predict(["apple"])

array(['joy'], dtype=object)